# Задание
Вопросы взяты из каждодневной практики, по ним вы можете оценить какого рода задачи у нас решают ML-разработчики.

Из базы загружены заказы клиентов магазина в форме двух pandas.DataFrame’ов: orders и order_lines.
    
Данные вида:

### orders 
содержит информацию о заказах: идентификатор заказа, идентификатор покупателя, время заказа

<br>OrderId,CustomerId,DateTime
<br>5,583,2017-01-01 15:03:17
<br>13,900,2019-02-05 05:02:59
<br>69,19573,2018-11-03 23:59:59


### order_lines 
содержит информацию о линиях чеков: идентификатор продукта, идентификатор заказа, цену продукта

<br>ProductId,OrderId,Price
<br>5873,5,3026.0
<br>7265,5,573.0
<br>9675,5,159.0
<br>5873,6,2999.0
<br>13,6,57.0

Постройте отчёт по популярным продуктами - функцию, возвращающую pandas.DataFrame, где видны
<br> - самые популярные за последний месяц продукты
<br> - суммарная выручка по каждому такому продукту
<br> - средний чек заказов, в которых есть такие продукты

In [1]:
import pandas as pd
import datetime

# вместо переменной datetime.datetime.today().month - текущий месяц, 
# возмем статическое значение месяца для простоты работы с сданными
month_curr = 11

In [2]:
# Загружаем данные в pandas DataFrame для последующей обработки

df1 = pd.read_excel('MindBox.xlsx', sheet_name='orders')
df2 = pd.read_excel('MindBox.xlsx', sheet_name='order_lines')

In [3]:
# Проверяем всё ли корректно загрузилось

df1.head()

,OrderId,CustomerId,DateTime
0,20,19,2017-01-01 15:03:17.000
1,25,25,2019-02-05 05:02:59.000
2,10,21,2018-11-03 23:59:59.000
3,21,15,2018-11-04 23:59:58.995
4,7,21,2018-11-05 23:59:58.995


In [4]:
# Проверяем всё ли корректно загрузилось

df2.head()

,ProductId,OrderId,Price
0,19,25,3026
1,27,12,573
2,12,8,159
3,24,15,2999
4,20,10,57


In [5]:
# Отбираем данные только за последний месяц (месяц = month_curr, константа для простоты работы с текущими данными)

df2_join = df2.join(df1, on='OrderId', how='left', lsuffix='OrderId')
df2_join = df2_join[df2_join['DateTime'].apply(lambda x: x.month == month_curr)]
df2_join.head()

,ProductId,OrderIdOrderId,Price,OrderId,CustomerId,DateTime
0,19,25,3026,13,20,2018-11-26 23:59:58.995
1,27,12,573,17,20,2018-11-13 23:59:58.995
2,12,8,159,18,25,2018-11-09 23:59:58.995
3,24,15,2999,9,22,2018-11-16 23:59:58.995
4,20,10,57,11,22,2018-11-11 23:59:58.995


In [6]:
# Популярные товары с точки зрения количества проданных товаров (продано больше трех штук)

df2_gr = pd.DataFrame(df2_join.groupby('ProductId')['Price'].count().sort_values(ascending=False))
df2_gr.head(10)

,Price
ProductId,
20,6
12,5
24,4
22,4
30,4
28,4
25,3
27,3
29,3


In [7]:
# Популярные товары с точки зрения количества проданных товаров (продано больше трех штук)

df2_gr = pd.DataFrame(df2_join.groupby('ProductId')['Price'].count().sort_values(ascending=False))
df2_most = df2_gr[df2_gr['Price'] > 3]
df2_most = df2_most.rename(index=str, columns={'Price':'counter'})
df2_most

,counter
ProductId,
20,6
12,5
24,4
22,4
30,4
28,4


In [8]:
# ProductId наиболее популярных товаров в виде списка

list_most = df2_most.index
list_most

Index(['20', '12', '24', '22', '30', '28'], dtype='object', name='ProductId')

In [9]:
#  Промежуточный DataFrame с группировкой по товарам и расчетом выручки по каждому товару

df2_sum = pd.DataFrame(df2_join.groupby('ProductId')['Price'].sum().sort_values(ascending=False))

In [10]:
# Суммарная выручка по каждому из популярных товаров

df2_sum = df2_sum[df2_sum.index.isin(list_most)]
df2_sum

,Price
ProductId,
20,16343
22,11254
12,10537
24,8905
28,8836
30,7623


In [11]:
# Получаем список чеков в которых есть товары из списка самых популярных товаров

list_order = df2_join[df2_join['ProductId'].isin(list_most)]['OrderId'].sort_values().unique()
list_order

array([ 8,  9, 11, 13, 14, 16, 17, 18, 19, 20, 24, 25], dtype=int64)

In [12]:
# Сумма каждого чека

df2_order = pd.DataFrame(df2_join.groupby('OrderId')['Price'].sum())
df2_order.head()

,Price
OrderId,
8,5539
9,9678
11,2160
13,11947
14,3891


In [13]:
# Получаем средний чек заказов в которых есть товары из сипска самых популярных товаров

mean_order = df2_order[df2_order.index.isin(list_order)].mean()
mean_order

Price    9671.5
dtype: float64

Отчет в виде функции, где ключ подаваемый на вход этой функции:
<br>1 - Cамые популярные за последний месяц продукты
<br>2 - Cуммарная выручка по каждому из наиболее популярных продуктов
<br>3 - Средний чек заказов, в которых есть наиболее популярные продукты

In [14]:
def report(type_report):
    if type_report in [1,2,3]:
        
        df1 = pd.read_excel('MindBox.xlsx', sheet_name='orders')
        df2 = pd.read_excel('MindBox.xlsx', sheet_name='order_lines')
        df2_join = df2.join(df1, on='OrderId', how='left', lsuffix='OrderId')
        df2_join = df2_join[df2_join['DateTime'].apply(lambda x: x.month == month_curr)]
        df2_gr = pd.DataFrame(df2_join.groupby('ProductId')['Price'].count().sort_values(ascending=False))
        df2_gr = pd.DataFrame(df2_join.groupby('ProductId')['Price'].count().sort_values(ascending=False))
        df2_most = df2_gr[df2_gr['Price'] > 3]
        df2_most = df2_most.rename(index=str, columns={'Price':'counter'})
        list_most = df2_most.index
        df2_sum = pd.DataFrame(df2_join.groupby('ProductId')['Price'].sum().sort_values(ascending=False))
        df2_sum = df2_sum[df2_sum.index.isin(list_most)]
        list_order = df2_join[df2_join['ProductId'].isin(list_most)]['OrderId'].sort_values().unique()
        df2_order = pd.DataFrame(df2_join.groupby('OrderId')['Price'].sum())
        mean_order = df2_order[df2_order.index.isin(list_order)].mean()
        
    if type_report == 1:
        return print('Cамые популярные за последний месяц продукты'), df2_most
    if type_report == 2:
        return print('Cуммарная выручка по каждому из наиболее популярных продуктов'), df2_sum
    if type_report == 3:
        return print('Средний чек заказов, в которых есть наиболее популярные продукты =', mean_order[0])
    return print('Ошибка ввода')
        

In [15]:
report(1)

Cамые популярные за последний месяц продукты


(None,            counter
 ProductId         
 20               6
 12               5
 24               4
 22               4
 30               4
 28               4)

In [16]:
report(2)

Cуммарная выручка по каждому из наиболее популярных продуктов


(None,            Price
 ProductId       
 20         16343
 22         11254
 12         10537
 24          8905
 28          8836
 30          7623)

In [17]:
report(3)

Средний чек заказов, в которых есть наиболее популярные продукты = 9671.5


In [18]:
report(4)

Ошибка ввода
